In [1]:
%load_ext autoreload
%autoreload 2

from util import *
from myalgorithm import algorithm

In [2]:
import os

file_name=  'STAGE1_1.json'

problem_file = os.path.join("stage1_problems", file_name)
timelimit = 30


with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time)

In [26]:
ALL_ORDERS[0].__dict__.keys()

dict_keys(['id', 'order_time', 'shop_lat', 'shop_lon', 'dlv_lat', 'dlv_lon', 'cook_time', 'volume', 'deadline', 'ready_time'])

In [65]:
ALL_RIDERS[0].__dict__

{'type': 'BIKE',
 'speed': 5.291005291005291,
 'capa': 100,
 'var_cost': 60,
 'fixed_cost': 8000,
 'service_time': 120,
 'available_number': 20,
 'T': array([[ 120.,  809., 1093., ..., 1703.,  784.,  730.],
        [ 809.,  120.,  898., ..., 1386.,  548.,  709.],
        [1093.,  898.,  120., ...,  731.,  498., 1403.],
        ...,
        [1703., 1386.,  731., ...,  120., 1072., 1952.],
        [ 784.,  548.,  498., ..., 1072.,  120., 1025.],
        [ 730.,  709., 1403., ..., 1952., 1025.,  120.]])}

In [44]:
import pandas as pd

ord_df = pd.DataFrame(columns=ALL_ORDERS[0].__dict__.keys())

for row in ALL_ORDERS:
    ord_df = pd.concat([ord_df, pd.DataFrame([vars(row)])])


ord_df

/var/folders/3l/z2_cjhyd20v1_yppby93tmnh0000gn/T/ipykernel_89132/4223044299.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ord_df = pd.concat([ord_df, pd.DataFrame([vars(row)])])


,id,order_time,shop_lat,shop_lon,dlv_lat,dlv_lon,cook_time,volume,deadline,ready_time
0,0,28,35.997628,125.982584,35.995128,125.977870,600,21,2176,628
0,1,86,36.012988,126.004387,36.004457,126.004163,900,12,2680,986
0,2,118,35.991074,126.022580,35.984552,126.029739,600,22,2419,718
0,3,155,36.021098,126.019330,36.016537,126.019030,600,44,2303,755
0,4,156,36.018153,125.972029,36.017555,125.986989,900,16,2882,1056
...,...,...,...,...,...,...,...,...,...,...
0,95,3466,35.991851,125.992472,35.988031,125.993688,600,13,5591,4066
0,96,3474,36.016543,125.996638,36.018567,125.998289,900,28,5843,4374
0,97,3493,35.990046,126.040079,35.988455,126.048018,600,27,5717,4093
0,98,3496,36.001260,126.004988,35.999274,126.010367,1200,33,6252,4696


In [ ]:
volume_eval = {}

ord_df = ord_df.sort_values(by="volume", ascending=False)

for ord in ord_df:

    volume_eval[ord.id] = 

In [59]:
sector = ord_df.order_time.quantile([0.3 *i for i in  range(1, 4)])
sector

0.3     774.4
0.6    1881.4
0.9    3287.8
Name: order_time, dtype: object